In [ ]:
#Import some libraries for general use
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [ ]:
#Import all of the bma_reference tools
from bma_reference import * 

In [ ]:
#Lets create a "loan tape" with 10 loans that we can use in examples.  
#We will use a python dictonary as the data storage object, but 
#you could just as easily read information into a pandas dataframe from csv or a json

"""
BMA Examples Loan Tape

This creates a loan_tape dictionary with all loans from BMA examples,
formatted with origination and current characteristics.

Loan IDs follow format: SF{page}_{examplenum}
Examples: SF4_01, SF7_01, SF12_01 (pool 1), SF12_02 (pool 2), etc.

Tape Format:
loan_tape = {
    'loan_id': {
        'original_balance': float,
        'gross_coupon': float,      # Annual percentage (e.g., 9.5 = 9.5%)
        'original_term': int,        # Months
        'origination_date': str or None,  # YYYY-MM-DD format
        'loan_age': int,             # WALA - months since origination
        'current_balance': float,
        'remaining_term': int,       # WAM - months remaining
        'asof_date': str or None,    # YYYY-MM-DD format
        'description': str,
    },
    ...
}
"""

loan_tape = {
    'SF4_01': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 9.5,          # Annual percentage
        'original_term': 360,         # Months
        'origination_date': None,
        'loan_age': 0,                # New loan, at origination
        'current_balance': 100_000,
        'remaining_term': 360,
        'asof_date': None,
        'description': 'A mortgage pass-through issued with a net coupon of 9.0%, a gross coupon of 9.5% and a term of 360 months. First month cash flow calculation (month 1).',
    },
    
    'SF7_01': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 9.5,          # GNMA I: 50bp servicing -> 9.0% net
        'original_term': 359,         # Months (359mo remaining at issue)
        'origination_date': '1988-03-01',
        'loan_age': 16,               # WALA at start of month 17
        'current_balance': 85_150.625,  # = 100_000 * 0.85150625
        'remaining_term': 344,        # 359 - 16 + 1 (before month 17 amortization)
        'asof_date': '1989-06-01',   # Beginning of month 17
        'description': 'Ginnie Mae I 9.0% pass-through issued 3/1/88. Back-calculating prepayment rates from observed factors for month 17.',
    },
    
    'SF12_01': {
        'original_balance': 1_000_000,  # $1M original face
        'gross_coupon': 9.5,            # GNMA I: 50bp servicing
        'original_term': 358,            # Months
        'origination_date': '1988-04-01',
        'loan_age': 9,                  # WALA at start of 6-month observation
        'current_balance': 869_252.18,  # = 1_000_000 * 0.86925218
        'remaining_term': 349,           # 358 - 9
        'asof_date': '1989-01-01',      # Start of 6-month observation
        'description': 'Pool 1 for SF-12: GNMA I 9%, orig 4/1/88, $1M original, 6-month observation',
    },
    
    'SF12_02': {
        'original_balance': 2_000_000,  # $2M original face
        'gross_coupon': 9.5,             # GNMA I: 50bp servicing -> 9.0% net
        'original_term': 360,            # Months
        'origination_date': '1988-12-01',
        'loan_age': 1,                  # WALA at start of 6-month observation
        'current_balance': 1_999_016.24,  # = 2_000_000 * 0.99950812
        'remaining_term': 359,           # 360 - 1
        'asof_date': '1989-01-01',      # Start of 6-month observation
        'description': 'Pool 2 for SF-12: GNMA I 9%, orig 12/1/88, $2M original, 6-month observation',
    },
    
    'SF23_01': {
        'original_balance': 100_000_000,  # $100M original face
        'gross_coupon': 8.0,               # WAC
        'original_term': 360,               # Months (WAM)
        'origination_date': None,
        'loan_age': 0,                     # At origination
        'current_balance': 100_000_000,
        'remaining_term': 360,
        'asof_date': None,
        'description': 'BMA Cash Flow A (SF-28): 8.00% WAC, 360 months WAM, $100M original balance, 1% SMM, 1% MDR',
    },
    
    'SF31_01': {
        'original_balance': 100_000_000,  # $100M original face
        'gross_coupon': 8.0,               # WAC
        'original_term': 360,              # Months (WAM)
        'origination_date': None,
        'loan_age': 0,                     # At origination
        'current_balance': 100_000_000,
        'remaining_term': 360,
        'asof_date': None,
        'description': 'BMA Cash Flow B (SF-29): 8.00% WAC, 360 months WAM, $100M original balance, 100% PSA, 100% SDA',
    },
    
    'SF42_01': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 8.0,          # Annual percentage (assumed typical)
        'original_term': 360,         # Months (assumed standard 30-year)
        'origination_date': '1993-07-01',
        'loan_age': 15,               # WALA at start of month 16
        'current_balance': 96_891.577,  # = 100_000 * 0.96891577
        'remaining_term': 341,        # WAM = 341 months (given)
        'asof_date': '1993-09-01',    # September 1993
        'description': 'Fannie Mae pool issued July 1993. WAM/WALA not directly reported, must be inferred from pool type and elapsed time.',
    },
    
    'SF42_02': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 7.5,          # Given in example
        'original_term': 360,        # Standard 30-year
        'origination_date': '1993-05-01',
        'loan_age': 4,                # WALA at start: 4 (before month 5)
        'current_balance': 97_000.0,   # = 100_000 * 0.970000
        'remaining_term': 355,        # = 359 - 4 (adjusted for reporting lag)
        'asof_date': '1993-09-01',    # September 1993 factor date
        'description': 'Ginnie Mae pool issued May 1993. WAM 359mo, WALA 1mo reported Oct 1993. Adjusted for 4-month reporting lag → effective age=5.',
    },
    
    'SF49_01': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 9.5,          # Annual percentage
        'original_term': 360,         # Months
        'origination_date': None,
        'loan_age': 0,                # At origination
        'current_balance': 100_000,
        'remaining_term': 360,
        'asof_date': None,
        'description': 'Yield/Duration calculation example: 9.0% net coupon, 360 months, 150% PSA',
    },
    
    'SF51_01': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 9.5,          # Annual percentage
        'original_term': 360,         # Months
        'origination_date': None,
        'loan_age': 0,                # At origination
        'current_balance': 100_000,
        'remaining_term': 360,
        'asof_date': None,
        'description': 'Yield/Duration calculation example: 9.0% net coupon, 360 months, 200% PSA',
    },
    
    'SF56_01': {
        'original_balance': 100_000,  # Normalized from 1.0 (BMA example uses normalized values)
        'gross_coupon': 10.0,         # Annual percentage
        'original_term': 3,           # Months (3-period example)
        'origination_date': None,
        'loan_age': 0,                # At origination
        'current_balance': 100_000,
        'remaining_term': 3,
        'asof_date': None,
        'description': 'Average life convention example: 3-period loan, demonstrates GPM/ARM vs Z-bond conventions',
    },
}


In [ ]:
#Lets run a few of the examples in the BMA guide to show how our funcitons work:

